In [62]:
library(tidyverse)
library(tidymodels)
library(janitor)
library(leaps)
library(GGally)

# STAT 306 Group C3 Project

# The Data & Goal Analysis

The data being explored is the [Real Estate Valuation Data Set](https://archive.ics.uci.edu/dataset/477/real+estate+valuation+data+set) that explores real-estate prices in Sindian Dist., New Taipei City, Taiwan. This data consists of n=414 observations that contain various numerical and time-related features. This data was obtained through UC Irvine Machine Learning Repository.

The variables we have are: 
- X1 =  The transaction date in numerical units. For instance, 2013.250 equals March 2013, where the month is depicted as a fraction of the year (3/12 = 0.250) 
- X2 = The house age in years
- X3 = Distance to the nearest MRT station in metres (MRTs are metro transit systems) 
- X4 = Number of convenience stores in the living circle on foot by count (integer)
- X5 = Latitude in degree
- X6 = Longitude in degree

**The primary objective of this analysis is to determine how the real estate price is influenced by various factors such as house age, proximity to transportation (MRT), convenience store availability, and geographical location.**

# Reading In the Data

In [63]:
real_estate_data <- read_csv("./data/real_estate.csv")
real_estate_data <- clean_names(real_estate_data)
head(real_estate_data)

set.seed(123) # for reproducibility
df_split <- initial_split(real_estate_data, prop = 0.8, strata = y_house_price_of_unit_area)
df_train <- training(df_split)
df_test <- testing(df_split)

Rows: 414 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (8): No, X1 transaction date, X2 house age, X3 distance to the nearest M...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


no,x1_transaction_date,x2_house_age,x3_distance_to_the_nearest_mrt_station,x4_number_of_convenience_stores,x5_latitude,x6_longitude,y_house_price_of_unit_area
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2012.917,32.0,84.87882,10,24.98298,121.5402,37.9
2,2012.917,19.5,306.59470,9,24.98034,121.5395,42.2
3,2013.583,13.3,561.98450,5,24.98746,121.5439,47.3
4,2013.500,13.3,561.98450,5,24.98746,121.5439,54.8
5,2012.833,5.0,390.56840,5,24.97937,121.5425,43.1
6,2012.667,7.1,2175.03000,3,24.96305,121.5125,32.1


# Exploratory Data Analysis

To begin our EDA, we first determine the severity of our missing data

In [64]:
na_count <- data.frame(sapply(df_train, function(y) sum(is.na(y))))
na_count

,sapply.df_train..function.y..sum.is.na.y...
,<int>
no,0
x1_transaction_date,0
x2_house_age,0
x3_distance_to_the_nearest_mrt_station,0
x4_number_of_convenience_stores,0
x5_latitude,0
x6_longitude,0
y_house_price_of_unit_area,0


As determined in the above code cell, there are no observations with missing data; we may proceed without any data imputation.



As per the objective of this project, we explore house age, distance to the nearest mrt station, number of convenience stores, and geographical location in relation to the response variable: the house price of unit area. The transaction date feature was dropped as it incorporates a time series component; this greatly increases the complexity of the task and demands the assertion of several strong assumptions for any model within the scope of STAT 306.

To create a feature for geographical location, we must discretize the longitude and latitude features together; this allows for a simpler analysis. We use k-means clustering to combine the two features into one.

In [65]:
# Create location-based features
df_train$neighborhood <- kmeans(df_train[, c('x5_latitude', 'x6_longitude')], centers = 5)$cluster 

A simple next step would be to explore the basic summary statistics of our data.

In [66]:
df_train %>%
    select(
        x2_house_age,
        x3_distance_to_the_nearest_mrt_station,
        x4_number_of_convenience_stores,
        y_house_price_of_unit_area
    ) %>%
    summary()

  x2_house_age   x3_distance_to_the_nearest_mrt_station
 Min.   : 0.00   Min.   :  23.38                       
 1st Qu.: 8.50   1st Qu.: 289.32                       
 Median :15.90   Median : 492.23                       
 Mean   :17.52   Mean   :1087.99                       
 3rd Qu.:27.60   3rd Qu.:1455.80                       
 Max.   :43.80   Max.   :6488.02                       
 x4_number_of_convenience_stores y_house_price_of_unit_area
 Min.   : 0.000                  Min.   :  7.60            
 1st Qu.: 1.000                  1st Qu.: 27.70            
 Median : 4.000                  Median : 38.50            
 Mean   : 4.009                  Mean   : 38.01            
 3rd Qu.: 6.000                  3rd Qu.: 46.60            
 Max.   :10.000                  Max.   :117.50            

In [ ]:
df_train %>%
    ggplot(aes(x = x6_longitude, y = x5_latitude, colour = as.factor(neighborhood), shape = as.factor(neighborhood))) +
        geom_point() + 
        xlab("Longitude") + 
        ylab("Latitude") +
        ggtitle("Neighbourhood cluster assignments (for reference)")

In [ ]:
df_train %>%
    ggplot(aes(x = x2_house_age, y = y_house_price_of_unit_area)) +
        geom_point() + 
        xlab("House Age") + 
        ylab("House Price") +
        ggtitle("House Price vs House Age")

From the above graph, we can determine that there exists a some relationship between house age and price. It can be observed that prices lower as houses get older, however, at around 30 years, the price of these houses can remain high. This could potentially be explained by the types of houses that have lasted for over 30 years. For example, apartment buildings might have been demolished after 30 years to build new ones whereas single family homes may have been maintained for much longer. This could be supported by the existence of fewer observations for houses older than 30 years

In [ ]:
df_train %>%
    ggplot(aes(x = x3_distance_to_the_nearest_mrt_station, y = y_house_price_of_unit_area)) +
        geom_point() + 
        xlab("Distance to nearest MRT station") + 
        ylab("House Price") +
        ggtitle("House Price vs Distance to nearest MRT station")

From the above graph, it seems there is a strong correlation between a house's distance to the nearest MRT station and its price. Homes closer to MRT stations are likely ones within a densely populated area; this would likely bring up the price

In [ ]:
df_train %>%
    ggplot(aes(x = as.factor(x4_number_of_convenience_stores), y = y_house_price_of_unit_area)) +
        geom_boxplot() + 
        xlab("Number of convenience stores") + 
        ylab("House Price") +
        ggtitle("House Price vs Number of convenience stores")

Since the number of convenience store is represented as an integer, the most appropriate visualization would be through a box plot. This demonstrates that housing prices tend to increase as the number of convenience stores increase. This is likely a result of the appeal of many nearby convenience stores which can offer a variety of shopping and food options.

In [ ]:
df_train %>%
    ggplot(aes(x = as.factor(neighborhood), y = y_house_price_of_unit_area)) +
        geom_boxplot() + 
        xlab("Neighborhood") + 
        ylab("House Price") +
        ggtitle("House Price vs Neighborhood")

Since we represent locational data as clusters, it is inherently a categorical variable. This boxplot demonstrates that there seems to be some indication that specific neighborhoods/regions have higher priced houses compared to others. This could be a result of major cities/neighbourhoods having a higher cost of living compared to non-major cities/neighbourhoods; for example, in BC, a similar pattern can be seen between Metro Vancouver and the Okanagan.

To further explore our data, we can create a correlation matrix to identify any potentially collinear features. We do not include neighbourhood here it is a categorical feature

In [ ]:
df_train %>%
    mutate(y = y_house_price_of_unit_area,
        house_age = x2_house_age,
        distance_to_mrt = x3_distance_to_the_nearest_mrt_station,
        num_convenience_stores = x4_number_of_convenience_stores,
        neighborhood = as.factor(neighborhood)
    ) %>%
    select(
        y,
        house_age,
        distance_to_mrt,
        num_convenience_stores
    ) %>%
    ggcorr(label=TRUE)

As observed in the above correlation matrix, there exists a collinear relationship between distance to mtr and the number of convenience stores.

We may now explore the distribution of our response variable $y$ (House price of unit area)

In [ ]:
df_train %>%
    ggplot(aes(x=y_house_price_of_unit_area)) +
        geom_histogram() +
        xlab("House Price") +
        ylab("Count") +
        ggtitle("Histogram of House Prices")

The data seems to be relatively normally distributed. That being said, there exists an outlier that may be of interest to explore. This outlier has a price that is significantly beyond that of the rest of the data with a house price of unit area at $117.5$.

This observation is quite close compared to the mean distance ($\bar x = 1087.988, x_i = 252.582$)to its nearest mrt station which partially explains its high price. However it is close to only one convenience store (average is 4), this goes against the initial data analysis where a greater number convenience stores suggests a greater expected price. It is important to also note that the grand mean of house prices is $y=38.012$ whereas the mean house price within this house's neighbourhood (5) is 43.418.

In a more advanced attempt at this project, stratification and cluster sampling schemes could be employed to obtain a better understanding of the data

In [ ]:
df_train[df_train$y_house_price_of_unit_area > 100, c(4,5,8,9)]

In [ ]:
mean(df_train$x3_distance_to_the_nearest_mrt_station)
mean(df_train$x4_number_of_convenience_stores)
mean(df_train$y_house_price_of_unit_area)
mean(df_train[df_train$neighborhood == 5,]$y_house_price_of_unit_area)

# Feature Engineering & Model Development

In [ ]:
# Proximity-based features
df_train$distance_to_mrt_category <- cut(df_train$x3_distance_to_the_nearest_mrt_station,
                            breaks = c(-Inf, 250, 500, 750, 1000, Inf),
                            labels = c("under_250m", "250m_500m", "500m_750m", "750m_1000m", "over_1000m"))

model_df <- select(df_train, -c(no))

## Model Selection

In [ ]:
s <- regsubsets(y_house_price_of_unit_area ~.-x3_distance_to_the_nearest_mrt_station -x1_transaction_date, data = model_df, method = "exhaustive", nvmax = 100)
ss <- summary(s)

rss <- ss$rss
adjr2 <- ss$adjr2
bic <- ss$bic

variables <- data.frame((ss$which))
variables$model <- rownames(variables)
rownames(variables) <- 1:nrow(variables)

rss <- ss$rss
adjr2 <- ss$adjr2
bic <- ss$bic

output <- cbind(variables, RSS=rss, AdjR2=adjr2, BIC=bic)

best_adjr2 <- which.max(output$AdjR2) 
print(best_adjr2)
print(max(output$AdjR2))
output %>% slice(best_adjr2)

## Interpretation Of The Results

## Results Validation
### Hypothesis Testing

In [ ]:
model_sum <- summary(reg)
p_values<- coef(model_sum)[, "Pr(>|t|)"]
tidy_p_values <- data.frame(P_Value = p_values)
tidy_p_values

A low p value (p $\leq$ 0.05 by default) suggests the variable is statistically significant. In the best model we selected, variables like `x2_house_age`, `distance_to_mrt_category`, `x5_latitude` and the intercept all have p value below 0.05. It indicates that these variables are statistically significant, while `x4_number_of_convenience_stores`, ` x6_longitude` and `neighborhood` are not. Nevertheless, the the model as a whole is significant because the p value of F statistic is far below 0.05. It indicates these selected predictors have effect on the response variable `y_house_price_of_unit_area` collectively.

We are not supposed to remove the variables that are less significant (p > 0.05) at this stage because this would consist of using the data to find significant variables. We could not fully trust the inference quantities (Std. Error and t-values) in the re-fitted model. All components of variable selection should be done in the `model_df` data set so that we can have an accurate estimate of the uncertainties in price inference.